In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import date
import datetime as dt
import pandas as pd
import json
import pprint
import sqlite3
import random
import string
import logging
import logging.handlers
import os
import time
from logging.handlers import RotatingFileHandler
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from CMC import refresh_CMC_API
from query import query
from Yahoo_API import yahoo_refresh
from ranking import ranking_f

#cwd = os.getcwd()

           

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler('logs_file',  when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)
secretKey = "SIMOSIUDSFYODS&TTFS987f9ds7f8sd6DFOUFYWE&FY"


    
def checkAuth():
    
    try:
    
        got_login = request.get_cookie("user", secret=secretKey)
        randStr = request.get_cookie("randStr", secret=secretKey)
        log.info(str(got_login) + ' ' + request.method + ' ' +
                 request.url + ' ' + request.environ.get('REMOTE_ADDR'))
        if (got_login in users)  and (users[got_login]["loggedIn"] == True):
        #users[got_login]["lastSeen"] = time.time()
           return got_login
        else:
            return redirect('/login') 
    except:
        return redirect('/login')
    

def query (query):
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    c.execute(query)
    result = c.fetchall()
    conn.close()
    return result


    
def convert_dict(tupl, dic):
    for a, b in tupl:
        dic.setdefault(a,[]).append(b)
    return dic


def ref_wallet():
    
    login = checkAuth()
    
    refresh_CMC_API()

    conn = sqlite3.connect('CryptoDB.db')
    
    c = conn.cursor()
    c.execute("select t.user_id, t.crypto_symbol, sum(t.quantity) as quantity, case when t.crypto_symbol = 'USD' then sum(t.quantity)*1  when t.crypto_symbol != 'USD' then sum(t.quantity)*a.price end as value from transactions t left join users u on u.user_id = t.user_id left join actual_data a on a.crypto_symbol = t.crypto_symbol group by t.user_id, t.crypto_symbol")
    wallet = c.fetchall()
    for coin in wallet:
        c.execute("Insert into wallets values (?,?,?,?,?)",(str(coin[0]), str(coin[1]), str(coin[2]), str(coin[3]), dt.datetime.now()))
    conn.commit()
    conn.close() 
                                                      
    return True

app = Bottle()


@app.get(r'/static/css/<filepath:re:.*\.css>')
def css(filepath):
    return static_file(filepath, root="static/css")

@app.get(r'/static/img/<filepath:re:.*\.(jpg|png|gif|ico|svg)>')
def img(filepath):
    return static_file(filepath, root="static/img")

@app.get(r'/static/js/<filepath:re:.*\.js>')
def js(filepath):
    return static_file(filepath, root="static/js")

@app.route('/adminrefresh')
def refresh():
    yahoo_refresh()
    return redirect('/')

@app.route('/')
@app.route('/index')
@app.route('/index/')
def base():
    return template('home')

@app.route('/home')
def home():
    return template('home')

@app.get('/login')
@app.get('/login/')
def login():
    return template('login')


@app.post('/login')
@app.post('/login/')
def login_rec(): 
    
    
    db_users = query('Select username,password from Users order by user_id')
    dictionary = {}
    db_users_dict = convert_dict(db_users, dictionary)

    
    got_login = request.forms.get('login', default=False)
    got_password = request.forms.get('password', default=False)
    
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    log.info(str(got_login) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    
    if (got_login in db_users_dict) and db_users_dict[got_login][0] == got_password:
        
        global users
        users={got_login:
        {"name":got_login, 
        "password": got_password, 
         "loggedIn":False,  "randStr":"",
         "lastSeen":0}
           } 
    
        response.set_cookie("user", got_login, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        users[got_login]["loggedIn"] = True
        users[got_login]["randStr"] = randStr
        users[got_login]["lastSeen"] = time.time()
        
        redirect('/mywallet')
        return (users)
    return template('login')

@app.route('/logout/')
@app.route('/logout')

def logout():
    global users
    users={"":
    {"name":"", 
    "password": "", 
    "loggedIn":False,  "randStr":"",
    "lastSeen":0}
    } 
    
    return redirect('/')


@app.get('/createAccount')
def register_get():
  
   return template('createAccount')


@app.post('/createAccount')
def register_post():   
    
    
    logins_emails = query('Select username, email from Users')
    logins_db = []
    emails_db = []
    for i in logins_emails:
        logins_db.append(i[0])
        emails_db.append(i[1])        
    login = request.forms.get('login')
    password = request.forms.get('password')
    email = request.forms.get('email')
    
    if(login in logins_db or email in emails_db):
        return redirect('/createAccount')
    else: 
        
        conn = sqlite3.connect('CryptoDB.db')
        c = conn.cursor()
        c.execute("Insert into USERS values(null,?,?,?,?)",(str(login),str(password),str(email),date.today()))
        conn.commit()
        created_user_id = query("select max(user_id) from users")
        c.execute("Insert into TRANSACTIONS values (null, ?,?,'USD', '1', '10000', '10000')", (str(created_user_id[0][0]),date.today()))
        conn.commit()
        conn.close() 

        
        return redirect('/login')
    
@app.route('/ranking')
def ranking():
    got_login = checkAuth()
    ref_wallet()
    df_ranking = ranking_f(got_login)
    
    return template('ranking', data = df_ranking)
    
@app.route('/prices')
def prices():
    
    refresh_CMC_API()
    table1 = query('SELECT C.NAME, A.PRICE, A.VOLUME_24H, A.MARKET_CAP FROM ACTUAL_DATA A LEFT JOIN CRYPTO_DICT C ON C.CRYPT_SYMBOL = A.CRYPTO_SYMBOL')
    df = pd.DataFrame(table1, columns = ['name', 'price', 'volume', 'market_cap']).to_html(justify = "center", classes = "table table-dark table-hover", index = True)
    ref_wallet()
                                                                             
    return template('prices', data = df)

@app.get('/graph')
@app.get('/graph/')
def choice(): 
    
    got_login = checkAuth()
    conn = sqlite3.connect('CryptoDB.db')

    c = conn.cursor()
    c.execute('select distinct c.name from historical_data h left join crypto_dict c on c.crypt_symbol = h.crypto_symbol where crypto_symbol != "nan" ')
    choice_list = c.fetchall()
    df_choice = pd.DataFrame(choice_list, columns = ['Name'])
    cryptos = df_choice['Name'].tolist()
    
    conn.close() 
    return template('list', title = 'crypto', cryptos = cryptos)
    

@app.post('/graph')
def graph():
    
    chosen = request.forms.get('chosen')
    
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    
    c.execute('select h.date, h.close from historical_data h left join crypto_dict c on c.crypt_symbol = h.crypto_symbol  where c.name = ?', (str(chosen),))
    chart = c.fetchall()
    df = pd.DataFrame(chart, columns = ['Date', 'price'])
    day = df['Date'].tolist()
    price = df['price'].tolist()
    conn.close() 
    return template('graph', day = day, price = price, title = str(chosen))

@app.get('/orderbefore')
def orderbefore_get():
    checkAuth()
    cryptos = query('SELECT CRYPTO_SYMBOL FROM ACTUAL_DATA')
    df = pd.DataFrame(cryptos, columns = ['Crypto'])
    cryptos = df['Crypto'].tolist()
    return template('orderbefore', cryptos = cryptos)

@app.post('/orderbefore')
def orderbefore_post():
    got_login = checkAuth()
    
    global crypto_chosen
    crypto_chosen = request.forms.get('chosen')
    redirect('/order')
    return (crypto_chosen)

@app.get('/order')
def order_get():
    checkAuth()
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    c.execute('SELECT PRICE FROM ACTUAL_DATA WHERE CRYPTO_SYMBOL = ?',(str(crypto_chosen),))
    price = c.fetchall()
    conn.close() 
    return template('order', crypto = crypto_chosen, prize = price[0][0])

@app.post('/order')
def order_post():

    got_login = checkAuth()
    ref_wallet()
    quantity = request.forms.get('quantity')
    #crypto = request.forms.get('chosen')
    orderType = request.forms.get('orderType')
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    c.execute('SELECT PRICE FROM ACTUAL_DATA WHERE CRYPTO_SYMBOL = ?',(str(crypto_chosen),))
    price_lst = c.fetchall()
    price = price_lst[0][0]
    amount = price*float(quantity)
    
    c = conn.cursor()
    c.execute("select w.CRYPTO_SYMBOL, w. quantity from wallets w left join users u on u.user_id = w.user_id where strftime('%Y-%m-%d %H:%M:%S', w.createddate)>= (select distinct strftime('%Y-%m-%d %H:%M:%S',max(ww.createddate)) from wallets ww left join users uu on uu.user_id = ww.user_id where uu.username = ?) and u.username = ? ; ", (str(got_login),(str(got_login) )))
    lst = c.fetchall()
    
    cryptos = []
    quant = []

    for x in lst:
        cryptos.append(x[0])
        quant.append(x[1])

    assets_dict = dict(zip(cryptos, quant))
    
    if (orderType == 'buy'):
        if(float(amount) <= float(assets_dict['USD'])):
            c = conn.cursor()

            c.execute("select user_id from users where username = ?", (str(got_login), ))
            user_id_lst = c.fetchall() 
            user_id = user_id_lst[0][0]                            
            c.execute("Insert into TRANSACTIONS values (null, ?,?,?,?,?, ?)", (str(user_id), 
                                                                                                   dt.datetime.now(),
                                                                                                  str(crypto_chosen),
                                                                                                    str(price),
                                                                                                  str(amount*-1),
                                                                                                  str(quantity)
                                                                                                  ))
            conn.commit()
            c.execute("Insert into TRANSACTIONS values (null, ?,?,'USD','1',?, ?)", (str(user_id), 
                                                                                                   dt.datetime.now(),
                                                                                                  str(amount),
                                                                                                  str(amount*-1)
                                                                                                  ))
            conn.commit()
            conn.close() 
            
        else:
            return template('error')
            
    else:
        
        try:
            if(float(quantity) <= float(assets_dict[crypto_chosen])):

                c = conn.cursor()

                c.execute("select user_id from users where username = ?", (str(got_login), ))
                user_id_lst = c.fetchall() 
                user_id = user_id_lst[0][0]                            
                c.execute("Insert into TRANSACTIONS values (null, ?,?,?,?,?, ?)", (str(user_id), 
                                                                                                       dt.datetime.now(),
                                                                                                      str(crypto_chosen),
                                                                                                        str(price),
                                                                                                      str(amount),
                                                                                                      str(float(quantity)*-1)
                                                                                                      ))
                conn.commit()
                c.execute("Insert into TRANSACTIONS values (null, ?,?,'USD','1',?, ?)", (str(user_id), 
                                                                                                       dt.datetime.now(),
                                                                                                      str(amount),
                                                                                                      str(amount)
                                                                                                      ))
                conn.commit()
                conn.close() 
            else:
                return template('error')
        except:
            return template('error')
    
    ref_wallet()
    return redirect('/')

@app.route('/mywallet')
def my_wallet():
    
    login_got = checkAuth()
    ref_wallet()
    #pie chart
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    
    c.execute("select case when w.crypto_symbol = 'USD' then 'USD' when w.crypto_symbol != 'USD' then c.name end as asset, sum(w.value)  from wallets w left join crypto_dict c on c.crypt_symbol = w.crypto_symbol left join users u on u.user_id = w.user_id  where u.username = ?  and strftime('%Y-%m-%d %H:%M:%S',w.createddate) = strftime('%Y-%m-%d %H:%M:%S',(select max(ww.createddate) from wallets ww) ) group by crypto_symbol", (str(login_got),))
    pie_chart_list = c.fetchall()
    assets_list = []
    value_list = []
    for crypto in pie_chart_list:
            assets_list.append(crypto[0])
            value_list.append(crypto[1])
    
    
    c.execute("select strftime('%Y-%m-%d %H:%M:%S', y.date) as date, y.value from (select strftime('%Y-%m-%d %H:%M:%S',x.w_date) as date, x.wallet_value as value, rank () OVER ( PARTITION BY  strftime('%Y-%m-%d %H:%M:%S', x.w_date) ORDER BY x.w_date desc) as rank  from (select  w.createddate as w_date , sum(w.value) as wallet_value from wallets w left join users u on u.user_id = w.user_id where u.username = ? group by  strftime('%Y-%m-%d %H:%M:%S',w.createddate) order by 1 desc) x ) y where y.rank = 1",(str(login_got),) )
    line_chart_lst = c.fetchall()
    date_list2 = []
    value_list2 = []
    for crypto in line_chart_lst:
        date_list2.append(crypto[0])
        value_list2.append(crypto[1])
    
    
    c.execute("SELECT c.name, t.price, t.quantity, t.amount, strftime('%Y-%m-%d %H:%M:%S',t.date) FROM TRANSACTIONS t LEFT JOIN Users u ON u.user_id = t.user_id LEFT JOIN Crypto_Dict c ON c.crypt_symbol = t.crypto_symbol WHERE t.crypto_symbol != 'USD' AND u.username = ?",(str(login_got),))
    table1_lst = c.fetchall()
    df = pd.DataFrame(table1_lst, columns = ['Cryptocurrency', 'Price', 'Quantity', 'Amount', 'Date']).to_html(justify = "center", classes = "table table-dark table-hover", index = True)
    conn.close() 
    return template('mywallet', values = value_list, labels = assets_list, day = date_list2, price = value_list2, data = df)
    
    
    
    
run(app, host = 'localhost', port = 8080, debug = True)

C:\Users\leski\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\leski\anaconda3\lib\tokenize.py:528: ResourceWarning: unclosed <socket.socket fd=1388, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8080)>
  spos, epos, pos = (lnum, start), (lnum, end), end
Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [28/Dec/2021 23:33:17] "GET / HTTP/1.1" 200 4191
127.0.0.1 - - [28/Dec/2021 23:33:19] "GET /login HTTP/1.1" 200 2830
127.0.0.1 - - [28/Dec/2021 23:33:23] "POST /login HTTP/1.1" 303 0
<ipython-input-2-21427e9b77ae>:78: ResourceWarning: unclosed <ssl.SSLSocket fd=860,

In [11]:
import sqlite3 

got_login = 'JarekLeski'
conn = sqlite3.connect('CryptoDB.db')
c = conn.cursor()
c.execute("select w.CRYPTO_SYMBOL, w. quantity from wallets w left join users u on u.user_id = w.user_id where strftime('%Y-%m-%d %H:%M:%S', w.createddate)>= (select distinct strftime('%Y-%m-%d %H:%M:%S',max(ww.createddate)) from wallets ww left join users uu on uu.user_id = ww.user_id where uu.username = ?) and u.username = ? ; ", (str(got_login),(str(got_login) )))
lst = c.fetchall()
#conn.close() 
cryptos = []
quant = []

for x in lst:
    cryptos.append(x[0])
    quant.append(x[1])

assets_dict = dict(zip(cryptos, quant))



1.15